# 🧠 Multi-Layer Perceptron (MLP) для табличных данных

**Phase 2: Deep Learning Basics - Step 1**

---

## 🎯 Цели ноутбука

1. **Понять архитектуру MLP** и backpropagation
2. **Реализовать MLP на PyTorch** для табличных данных
3. **Сравнить с классическими методами** (XGBoost из Phase 1)
4. **Изучить ключевые техники:** dropout, batch normalization, оптимизаторы
5. **Гиперпараметры:** learning rate, batch size, epochs, architecture

---

## 💼 Бизнес-задача: Titanic Survival Prediction

**Контекст:** Предсказание выживаемости пассажиров Титаника.

**Почему MLP для табличных данных?**
- 📊 **Нелинейные зависимости:** MLP может захватить сложные паттерны
- 🔗 **Feature interactions:** Автоматически через скрытые слои
- 🧪 **Ансамбли:** MLP + Gradient Boosting часто лучше

**Честное сравнение:**
- ❌ **MLP обычно ХУЖЕ XGBoost** для табличных данных
- ✅ **Но:** полезен в ансамблях и для обучения Deep Learning
- ✅ **Цель:** Понять основы, которые нужны для CNN, RNN, Transformers

---

## 📚 Часть 1: Теория Multi-Layer Perceptron

### 1.1 Что такое MLP?

**Multi-Layer Perceptron (MLP)** — это полносвязная (fully-connected) нейронная сеть.

#### Архитектура

**Слои:**
1. **Input Layer:** Входные признаки (например, Age, Sex, Fare для Титаника)
2. **Hidden Layers:** Один или несколько скрытых слоев
3. **Output Layer:** Выходной слой (1 нейрон для бинарной классификации)

**Математически:**

Для слоя $l$:

$$z^{(l)} = W^{(l)} a^{(l-1)} + b^{(l)}$$

$$a^{(l)} = \sigma(z^{(l)})$$

где:
- $W^{(l)}$ — матрица весов слоя $l$
- $b^{(l)}$ — вектор bias
- $a^{(l-1)}$ — активации предыдущего слоя
- $\sigma$ — функция активации

**Полная forward pass:**

$$
\begin{align}
a^{(0)} &= x \quad \text{(input)} \\
z^{(1)} &= W^{(1)} a^{(0)} + b^{(1)} \\
a^{(1)} &= \sigma(z^{(1)}) \\
z^{(2)} &= W^{(2)} a^{(1)} + b^{(2)} \\
a^{(2)} &= \sigma(z^{(2)}) \quad \text{(output)}
\end{align}
$$

---

### 1.2 Функции активации

**Зачем нужны?** Без нелинейной активации, многослойная сеть = линейная модель!

#### 1.2.1 Sigmoid

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

**Свойства:**
- Выход: $(0, 1)$
- ✅ Интерпретация как вероятность
- ❌ **Vanishing gradient** для больших $|x|$
- ❌ **Not zero-centered** (все производные положительны)

**Производная:**

$$\sigma'(x) = \sigma(x)(1 - \sigma(x))$$

---

#### 1.2.2 Tanh

$$\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$

**Свойства:**
- Выход: $(-1, 1)$
- ✅ **Zero-centered**
- ❌ **Vanishing gradient** для больших $|x|$

**Производная:**

$$\tanh'(x) = 1 - \tanh^2(x)$$

---

#### 1.2.3 ReLU (Rectified Linear Unit) ⭐

$$\text{ReLU}(x) = \max(0, x)$$

**Свойства:**
- ✅ **Простота вычислений**
- ✅ **Нет vanishing gradient** для $x > 0$
- ✅ **Sparse activations** (многие нейроны = 0)
- ❌ **Dying ReLU** проблема (нейроны "умирают" при $x \leq 0$)

**Производная:**

$$\text{ReLU}'(x) = \begin{cases} 1, & \text{if } x > 0 \\ 0, & \text{if } x \leq 0 \end{cases}$$

**Почему ReLU стал стандартом?**
- Faster convergence (до 6x быстрее, чем sigmoid/tanh)
- Simplicity
- Биологически правдоподобно

---

#### 1.2.4 Leaky ReLU

$$\text{LeakyReLU}(x) = \max(0.01x, x)$$

**Решает dying ReLU:**
- Малый градиент (0.01) для $x < 0$ вместо 0

---

#### 1.2.5 ELU (Exponential Linear Unit)

$$\text{ELU}(x) = \begin{cases} x, & \text{if } x > 0 \\ \alpha(e^x - 1), & \text{if } x \leq 0 \end{cases}$$

**Свойства:**
- ✅ Smooth везде
- ✅ Mean activations ближе к нулю
- ❌ Медленнее ReLU (экспонента)

---

#### Сравнение активаций

| Активация | Vanishing Gradient | Dying Units | Скорость | Рекомендация |
|-----------|-------------------|-------------|----------|-------------|
| **Sigmoid** | ❌ Да | ✅ Нет | Slow | Только output для binary |
| **Tanh** | ❌ Да | ✅ Нет | Slow | RNN (иногда) |
| **ReLU** | ✅ Нет | ❌ Да | ⚡ Fast | ⭐ Стандарт для hidden |
| **Leaky ReLU** | ✅ Нет | ✅ Нет | ⚡ Fast | Альтернатива ReLU |
| **ELU** | ✅ Нет | ✅ Нет | Slower | Когда нужна гладкость |

**Практическая рекомендация:**
1. **Default:** ReLU для всех скрытых слоев
2. **Если dying ReLU проблема:** Leaky ReLU или ELU
3. **Output layer (binary):** Sigmoid
4. **Output layer (multiclass):** Softmax
5. **Output layer (regression):** Linear (no activation)

---

### 1.3 Backpropagation и Gradient Descent

#### Loss Function

**Binary Cross-Entropy (для classification):**

$$L = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]$$

где:
- $y_i$ — истинная метка (0 или 1)
- $\hat{y}_i$ — предсказанная вероятность

**Mean Squared Error (для regression):**

$$L = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2$$

---

#### Backpropagation Algorithm

**Цель:** Вычислить $\frac{\partial L}{\partial W^{(l)}}$ для всех слоев.

**Chain rule:**

$$\frac{\partial L}{\partial W^{(l)}} = \frac{\partial L}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \cdot \ldots \cdot \frac{\partial z^{(l)}}{\partial W^{(l)}}$$

**Алгоритм:**

1. **Forward pass:** Вычислить $a^{(l)}$ для всех слоев
2. **Compute loss:** $L = \text{loss}(a^{(L)}, y)$
3. **Backward pass:** 
   - Вычислить $\delta^{(L)} = \frac{\partial L}{\partial z^{(L)}}$
   - Для $l = L-1, L-2, \ldots, 1$:
     $$\delta^{(l)} = (W^{(l+1)})^T \delta^{(l+1)} \odot \sigma'(z^{(l)})$$
   - Градиенты весов:
     $$\frac{\partial L}{\partial W^{(l)}} = \delta^{(l)} (a^{(l-1)})^T$$

4. **Update weights:**
   $$W^{(l)} \leftarrow W^{(l)} - \eta \frac{\partial L}{\partial W^{(l)}}$$

где $\eta$ — learning rate.

---

#### Gradient Descent варианты

**1. Batch Gradient Descent:**
- Используем **все** данные для вычисления градиента
- ✅ Stable convergence
- ❌ Slow для больших данных

**2. Stochastic Gradient Descent (SGD):**
- Используем **один** пример за раз
- ✅ Fast updates
- ❌ Noisy gradient, может не сойтись

**3. Mini-Batch Gradient Descent:** ⭐
- Используем **batch** примеров (обычно 32, 64, 128, 256)
- ✅ Баланс скорости и стабильности
- ✅ Эффективное использование GPU

---

### 1.4 Оптимизаторы

#### 1.4.1 SGD (Stochastic Gradient Descent)

$$W_{t+1} = W_t - \eta \nabla L(W_t)$$

**Проблемы:**
- ❌ Одинаковый learning rate для всех параметров
- ❌ Может застрять в локальных минимумах

---

#### 1.4.2 SGD with Momentum

$$
\begin{align}
v_t &= \beta v_{t-1} + \nabla L(W_t) \\
W_{t+1} &= W_t - \eta v_t
\end{align}
$$

где $\beta$ (обычно 0.9) — momentum coefficient.

**Эффект:**
- ✅ Ускорение в правильном направлении
- ✅ Подавление колебаний

---

#### 1.4.3 RMSprop (Root Mean Square Propagation)

$$
\begin{align}
E[g^2]_t &= \beta E[g^2]_{t-1} + (1-\beta) (\nabla L)^2 \\
W_{t+1} &= W_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} \nabla L
\end{align}
$$

**Эффект:**
- ✅ Адаптивный learning rate для каждого параметра
- ✅ Малые обновления для параметров с большими градиентами

---

#### 1.4.4 Adam (Adaptive Moment Estimation) ⭐

**Комбинация Momentum + RMSprop:**

$$
\begin{align}
m_t &= \beta_1 m_{t-1} + (1 - \beta_1) \nabla L \quad \text{(1st moment, momentum)} \\
v_t &= \beta_2 v_{t-1} + (1 - \beta_2) (\nabla L)^2 \quad \text{(2nd moment, RMSprop)} \\
\hat{m}_t &= \frac{m_t}{1 - \beta_1^t} \quad \text{(bias correction)} \\
\hat{v}_t &= \frac{v_t}{1 - \beta_2^t} \\
W_{t+1} &= W_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t
\end{align}
$$

**Default параметры:**
- $\beta_1 = 0.9$
- $\beta_2 = 0.999$
- $\epsilon = 10^{-8}$

**Почему Adam лучший?**
- ✅ Adaptive learning rates
- ✅ Работает out-of-the-box для большинства задач
- ✅ Меньше tuning, чем SGD

---

#### 1.4.5 AdamW

Adam + **decoupled weight decay:**

$$W_{t+1} = W_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t - \eta \lambda W_t$$

где $\lambda$ — weight decay coefficient.

**Улучшение:** Правильная L2 регуляризация (в Adam была ошибка).

---

#### Сравнение оптимизаторов

| Оптимизатор | Скорость сходимости | Tuning сложность | Рекомендация |
|-------------|-------------------|------------------|-------------|
| **SGD** | Slow | High (LR, momentum) | Когда нужна стабильность |
| **SGD + Momentum** | Medium | Medium | Computer Vision (иногда) |
| **RMSprop** | Fast | Low | RNN (исторически) |
| **Adam** | ⚡ Fast | ⭐ Very Low | **Default для большинства задач** |
| **AdamW** | ⚡ Fast | ⭐ Very Low | Transformers, modern DL |

**Практическая рекомендация:**
1. **Start with Adam** (lr=0.001)
2. Если переобучение → AdamW (weight_decay=0.01)
3. Для CV иногда SGD+Momentum лучше (но нужен tuning)

---

### 1.5 Регуляризация и Batch Normalization

#### 1.5.1 Dropout

**Идея:** Случайно "выключать" нейроны во время обучения.

**Алгоритм (training):**
```python
for each neuron:
    with probability p:
        set activation to 0
    else:
        keep activation / (1-p)  # scaling
```

**При inference:** All neurons active (no dropout).

**Математически:**

$$a_i^{\text{train}} = \frac{1}{1-p} \cdot a_i \cdot r_i$$

где $r_i \sim \text{Bernoulli}(1-p)$.

**Почему работает?**
- ✅ **Ensemble эффект:** Обучаем много "подсетей"
- ✅ **Предотвращение co-adaptation:** Нейроны не могут полагаться на конкретные другие
- ✅ **Regularization:** Уменьшает overfitting

**Типичные значения:**
- $p = 0.2$ для входного слоя
- $p = 0.5$ для скрытых слоев

---

#### 1.5.2 L1/L2 Regularization

**L2 (Weight Decay):**

$$L_{\text{total}} = L_{\text{data}} + \frac{\lambda}{2} \sum_i w_i^2$$

**Эффект:** Малые веса → проще модель → меньше overfitting.

**L1 (Lasso):**

$$L_{\text{total}} = L_{\text{data}} + \lambda \sum_i |w_i|$$

**Эффект:** Sparse weights (многие = 0) → feature selection.

---

#### 1.5.3 Batch Normalization

**Проблема:** Internal covariate shift — распределение активаций меняется во время обучения.

**Решение:** Нормализация активаций для каждого mini-batch.

**Алгоритм:**

Для batch $B = \{x_1, \ldots, x_m\}$:

$$
\begin{align}
\mu_B &= \frac{1}{m} \sum_{i=1}^m x_i \quad \text{(mean)} \\
\sigma_B^2 &= \frac{1}{m} \sum_{i=1}^m (x_i - \mu_B)^2 \quad \text{(variance)} \\
\hat{x}_i &= \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad \text{(normalize)} \\
y_i &= \gamma \hat{x}_i + \beta \quad \text{(scale and shift)}
\end{align}
$$

где $\gamma, \beta$ — learnable parameters.

**Преимущества:**
- ✅ **Faster training** (можно использовать больший learning rate)
- ✅ **Regularization эффект** (меньше нужен dropout)
- ✅ **Меньше чувствительность к инициализации**

**Где вставлять?**

```
Linear → BatchNorm → ReLU
```

(или иногда: Linear → ReLU → BatchNorm)

---

#### 1.5.4 Early Stopping

**Идея:** Остановить обучение, когда validation loss перестает улучшаться.

**Алгоритм:**
```python
best_val_loss = infinity
patience_counter = 0

for epoch in range(max_epochs):
    train(...)
    val_loss = validate(...)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model()
        patience_counter = 0
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        break  # Stop training
```

**Типичные значения:**
- `patience = 5-20` epochs

---

## Теория завершена! Переходим к практике 🚀

---

## 📊 Часть 2: Практическая реализация

### 2.1 Импорт библиотек

In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Baseline для сравнения
from xgboost import XGBClassifier

# Визуализация
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
print(f'PyTorch version: {torch.__version__}')

print('✅ Библиотеки загружены')

### 2.2 Загрузка и подготовка данных

In [ ]:
# Загрузка данных Титаника
import os

data_path = '../../data/titanic_train.csv'

if not os.path.exists(data_path):
    print('❌ Файл не найден! Используем альтернативный путь...')
    # Попробуем другой путь
    data_path = '../titanic/titanic_train.csv'
    if not os.path.exists(data_path):
        print('❌ Данные не найдены. Создайте файл или загрузите с Kaggle.')
else:
    df = pd.read_csv(data_path)
    print(f'✅ Данные загружены: {df.shape[0]:,} строк, {df.shape[1]} столбцов')
    print(f'Target: Survived')

In [ ]:
# Первый взгляд
df.head()

In [ ]:
# Подготовка признаков (простая версия для демонстрации)
# Выберем основные признаки

# Заполняем пропуски
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Создаем признаки
df['Sex'] = (df['Sex'] == 'male').astype(int)
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# One-hot для Embarked
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

# Выбираем признаки
features = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone'] + \
           [col for col in df.columns if 'Embarked_' in col]

X = df[features].values
y = df['Survived'].values

print(f'Features: {features}')
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'Survival rate: {y.mean():.1%}')

In [ ]:
# Train/Val/Test split
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=RANDOM_STATE, stratify=y_temp
)

print(f'Train: {X_train.shape[0]:,} samples')
print(f'Val: {X_val.shape[0]:,} samples')
print(f'Test: {X_test.shape[0]:,} samples')

In [ ]:
# Масштабирование (критично для нейросетей!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f'Scaled X_train mean: {X_train_scaled.mean():.4f}')
print(f'Scaled X_train std: {X_train_scaled.std():.4f}')
print('✅ Данные масштабированы (mean=0, std=1)')

In [ ]:
# Конвертация в PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train)

X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val)

X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test)

print(f'X_train_tensor shape: {X_train_tensor.shape}')
print(f'y_train_tensor shape: {y_train_tensor.shape}')

In [ ]:
# Создаем DataLoaders для mini-batch training
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f'Batch size: {batch_size}')
print(f'Number of batches (train): {len(train_loader)}')
print(f'Number of batches (val): {len(val_loader)}')

### 2.3 Определение MLP архитектуры

In [ ]:
# Простая MLP архитектура
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1=64, hidden_dim2=32, dropout_rate=0.3):
        super(SimpleMLP, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.fc3 = nn.Linear(hidden_dim2, 1)
    
    def forward(self, x):
        # Layer 1: Linear → BatchNorm → ReLU → Dropout
        x = self.fc1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        
        # Layer 2: Linear → BatchNorm → ReLU → Dropout
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        
        # Output: Linear → Sigmoid
        x = self.fc3(x)
        x = torch.sigmoid(x)
        
        return x

# Инициализация модели
input_dim = X_train_scaled.shape[1]
model = SimpleMLP(input_dim=input_dim)
model = model.to(device)

print(model)
print(f'\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f'Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

### 2.4 Обучение модели

In [ ]:
# Loss и optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f'Loss function: {criterion}')
print(f'Optimizer: {optimizer}')

In [ ]:
# Training функция
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device).unsqueeze(1)  # (batch_size, 1)
        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Метрики
        total_loss += loss.item() * X_batch.size(0)
        predicted = (outputs > 0.5).float()
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Validation функция
def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device).unsqueeze(1)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item() * X_batch.size(0)
            predicted = (outputs > 0.5).float()
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)
            
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy, all_preds, all_labels

print('✅ Training functions готовы')

In [ ]:
# Training loop
num_epochs = 100
patience = 10

train_losses = []
val_losses = []
train_accs = []
val_accs = []

best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

print('Начинаем обучение...')
print('='*60)

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc, _, _ = validate_epoch(model, val_loader, criterion, device)
    
    # Сохраняем метрики
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    # Print каждые 10 эпох
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        print(f'  Best Val Loss: {best_val_loss:.4f}, Patience: {patience_counter}/{patience}')
    
    # Early stopping
    if patience_counter >= patience:
        print(f'\nEarly stopping на epoch {epoch+1}')
        break

# Загружаем лучшую модель
model.load_state_dict(best_model_state)

print('='*60)
print(f'✅ Обучение завершено!')
print(f'Best validation loss: {best_val_loss:.4f}')
print(f'Total epochs: {epoch+1}')

### 2.5 Визуализация обучения

In [ ]:
# График обучения
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(train_losses, label='Train Loss', alpha=0.8)
ax1.plot(val_losses, label='Val Loss', alpha=0.8)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (BCE)')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy
ax2.plot(train_accs, label='Train Accuracy', alpha=0.8)
ax2.plot(val_accs, label='Val Accuracy', alpha=0.8)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'Final train accuracy: {train_accs[-1]:.4f}')
print(f'Final val accuracy: {val_accs[-1]:.4f}')
print(f'Overfitting: {(train_accs[-1] - val_accs[-1]):.4f}')

### 2.6 Оценка на test set

In [ ]:
# Оценка на test
test_loss, test_acc, test_preds, test_labels = validate_epoch(
    model, test_loader, criterion, device
)

# Конвертация в numpy
test_preds_np = np.array(test_preds).flatten()
test_labels_np = np.array(test_labels).flatten()
test_preds_binary = (test_preds_np > 0.5).astype(int)

# Метрики
test_precision = precision_score(test_labels_np, test_preds_binary)
test_recall = recall_score(test_labels_np, test_preds_binary)
test_f1 = f1_score(test_labels_np, test_preds_binary)
test_roc_auc = roc_auc_score(test_labels_np, test_preds_np)

print('📊 MLP Test Results:')
print('='*50)
print(f'  Test Loss: {test_loss:.4f}')
print(f'  Test Accuracy: {test_acc:.4f}')
print(f'  Precision: {test_precision:.4f}')
print(f'  Recall: {test_recall:.4f}')
print(f'  F1-score: {test_f1:.4f}')
print(f'  ROC-AUC: {test_roc_auc:.4f}')
print('='*50)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels_np, test_preds_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('MLP Confusion Matrix (Test Set)')
plt.show()

print(f'True Negatives: {cm[0,0]}')
print(f'False Positives: {cm[0,1]}')
print(f'False Negatives: {cm[1,0]}')
print(f'True Positives: {cm[1,1]}')

### 2.7 Сравнение с XGBoost (Phase 1)

In [ ]:
# XGBoost для сравнения
print('Обучаем XGBoost для честного сравнения...')

xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    random_state=RANDOM_STATE,
    verbosity=0
)

xgb_model.fit(X_train_scaled, y_train)

# Предсказания XGBoost
xgb_preds = xgb_model.predict(X_test_scaled)
xgb_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Метрики XGBoost
xgb_acc = accuracy_score(y_test, xgb_preds)
xgb_precision = precision_score(y_test, xgb_preds)
xgb_recall = recall_score(y_test, xgb_preds)
xgb_f1 = f1_score(y_test, xgb_preds)
xgb_roc_auc = roc_auc_score(y_test, xgb_proba)

print('✅ XGBoost обучен')

In [ ]:
# Сравнительная таблица
comparison = pd.DataFrame({
    'Model': ['MLP (PyTorch)', 'XGBoost'],
    'Accuracy': [test_acc, xgb_acc],
    'Precision': [test_precision, xgb_precision],
    'Recall': [test_recall, xgb_recall],
    'F1-score': [test_f1, xgb_f1],
    'ROC-AUC': [test_roc_auc, xgb_roc_auc]
})

print('\n' + '='*70)
print('🏆 СРАВНЕНИЕ: MLP vs XGBoost')
print('='*70)
print(comparison.to_string(index=False))
print('='*70)

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

metrics = ['Accuracy', 'F1-score', 'ROC-AUC']
for i, metric in enumerate(metrics):
    axes[i].bar(['MLP', 'XGBoost'], comparison[metric], alpha=0.7, edgecolor='black')
    axes[i].set_ylabel(metric)
    axes[i].set_title(f'{metric} Comparison')
    axes[i].set_ylim([0.7, 0.9])
    axes[i].grid(True, alpha=0.3, axis='y')
    
    # Добавляем значения
    for j, v in enumerate(comparison[metric]):
        axes[i].text(j, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.show()

## 🎯 Выводы

### Что мы изучили:

1. **MLP архитектура:**
   - Полносвязные слои (Linear)
   - Функции активации (ReLU, Sigmoid)
   - Batch Normalization
   - Dropout для регуляризации

2. **Обучение нейросетей:**
   - Backpropagation и gradient descent
   - Adam optimizer
   - Binary Cross-Entropy loss
   - Early stopping
   - Mini-batch training

3. **PyTorch основы:**
   - `nn.Module` для определения модели
   - `DataLoader` для batch processing
   - Training loop (forward → loss → backward → update)
   - GPU support

### MLP vs XGBoost для табличных данных:

**Типичные результаты:**
- MLP: Accuracy ~78-82%, ROC-AUC ~0.82-0.85
- XGBoost: Accuracy ~80-84%, ROC-AUC ~0.84-0.87

**Вывод:** XGBoost обычно **лучше** для табличных данных!

### Когда использовать MLP?

✅ **Используйте MLP когда:**
- Много данных (>100k примеров)
- Сложные нелинейные зависимости
- Нужен ensemble с Gradient Boosting
- Transfer learning (предобучение на смежной задаче)
- Изучение Deep Learning основ

❌ **НЕ используйте MLP когда:**
- Мало данных (<10k)
- Нужна интерпретируемость
- Ограниченные ресурсы (XGBoost быстрее)
- Production система (XGBoost проще deploy)

### Ключевые уроки:

1. **Scaling критичен!** Нейросети требуют нормализации (mean=0, std=1)
2. **Batch Normalization** ускоряет обучение
3. **Dropout** предотвращает overfitting
4. **Early stopping** экономит время и улучшает обобщение
5. **Learning rate** - критичный гиперпараметр
6. **Adam optimizer** работает хорошо out-of-the-box

### Следующие шаги:

1. **Эксперименты:**
   - Разные архитектуры (глубже, шире)
   - Другие функции активации (ELU, LeakyReLU)
   - Learning rate scheduling
   - Разные optimizers (SGD, AdamW)

2. **1D-CNN:** Convolutional layers для табличных данных
3. **Autoencoders:** Unsupervised learning, anomaly detection
4. **Transfer Learning:** Предобучение на других данных

---

## 🎉 Phase 2, Step 1 завершен!

Вы освоили основы Deep Learning с MLP. Теперь готовы к более сложным архитектурам! 🚀
